In [9]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [10]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_pastis.ipynb


In [11]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [12]:
from scipy import spatial
from sklearn.decomposition import PCA

In [13]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [14]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [15]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [16]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [17]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,927.84,0.00,0.02,0.00,0.00,-0.00,0.00
3,0.00,270.47,-0.00,-0.00,-0.00,0.00,0.00
4,0.02,-0.00,69.59,-0.00,-0.00,-0.00,-0.00
5,0.00,-0.00,-0.00,13.44,-0.00,0.00,-0.00
6,0.00,-0.00,-0.00,-0.00,2.38,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.42,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.05


In [18]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00006,0.00002,0.00005,-0.00005,0.00006
3,0.00000,1.00000,-0.00001,-0.00005,-0.00004,0.00001,0.00001
4,0.00006,-0.00001,1.00000,-0.00003,-0.00003,-0.00004,-0.00001
5,0.00002,-0.00005,-0.00003,1.00000,-0.00002,0.00001,-0.00001
6,0.00005,-0.00004,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00005,0.00001,-0.00004,0.00001,0.00002,1.00000,0.00005
8,0.00006,0.00001,-0.00001,-0.00001,0.00001,0.00005,1.00000


In [19]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.024524249930512044

In [20]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.23318712e+05 1.01646514e+04 6.45398521e+02 2.26121362e+01
 6.33059009e-01 1.47829768e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998526,-0.052806,-0.012305,-0.002363,-0.000416,-0.000075,-0.000008
1,0.052018,0.997047,-0.055593,-0.009925,-0.001754,-0.000308,-0.000037
2,0.015035,0.054291,0.997088,-0.050649,-0.008570,-0.001511,-0.000181
3,0.003595,0.012376,0.049363,0.996769,-0.061162,-0.010342,-0.001243
4,0.000840,0.002896,0.011253,0.059441,0.993986,-0.090638,-0.010397
5,0.000223,0.000763,0.002966,0.015346,0.087936,0.988658,-0.120738
6,0.000054,0.000185,0.000721,0.003727,0.021029,0.119293,0.992629


In [21]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.0014739 , 0.00295342, 0.00291156, 0.00323126, 0.00601427,
       0.01134199, 0.00737085])